In [1]:
%pip install ../..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=80317 sha256=0198327f3c27517af1b7f4a91196915ad46251999b422af97ce719360ffb0a03
  Stored in directory: /tmp/pip-ephem-wheel-cache-kwy4laea/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.group_inference import JTTInference
from spuco.models import model_factory 
from spuco.utils import Trainer
from torch.optim import SGD

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)
trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
trainer.train(1)

Epoch 0: 100%|██████████| 751/751 [00:04<00:00, 161.89batch/s, accuracy=100.0%, loss=0.00559] 


In [6]:
predictions = torch.argmax(trainer.get_trainset_outputs(), dim=-1).detach().cpu().tolist()
jtt = JTTInference(
    predictions=predictions,
    class_labels=trainset.labels
)

Getting Trainset Outputs: 100%|██████████| 751/751 [00:01<00:00, 392.24batch/s]


In [7]:
group_partition = jtt.infer_groups()

In [8]:
for key in group_partition.keys():
    print(key, len(group_partition[key]))

(0, 0) 47761
(0, 1) 243


In [9]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:  50%|█████     | 1/2 [00:02<00:02,  2.42s/it]

Group (0, 0) Test Accuracy: 100.0


Evaluating group-wise accuracy: 100%|██████████| 2/2 [00:02<00:00,  1.41s/it]

Group (0, 1) Test Accuracy: 0.0


{(0, 0): 100.0, (0, 1): 0.0}

In [10]:
from spuco.datasets import GroupLabeledDatasetWrapper

group_labeled_trainset = GroupLabeledDatasetWrapper(trainset, group_partition)

In [27]:
from torch.optim import SGD
from spuco.invariant_train import UpSampleERM, DownSampleERM, CustomSampleERM
from spuco.models import model_factory 

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

jtt_train = UpSampleERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    group_partition=group_partition,
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
jtt_train.train()

486


Epoch 0: 100%|██████████| 8/8 [00:00<00:00, 19.07batch/s, accuracy=26.31578947368421%, loss=1.61]


In [28]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:08,  2.78it/s]

Group (0, 0) Test Accuracy: 98.3451536643026


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:08,  2.69it/s]

Group (0, 1) Test Accuracy: 8.983451536643026


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:08,  2.54it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:10<00:00,  2.33it/s]

Group (4, 3) Test Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:10<00:00,  2.36it/s]

Group (4, 4) Test Accuracy: 0.5050505050505051


{(0, 0): 98.3451536643026,
 (0, 1): 8.983451536643026,
 (0, 2): 0.0,
 (0, 3): 67.84869976359339,
 (0, 4): 99.52718676122932,
 (1, 0): 0.0,
 (1, 1): 0.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.8,
 (2, 1): 63.733333333333334,
 (2, 2): 1.8666666666666667,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 32.49370277078086,
 (3, 2): 94.7103274559194,
 (3, 3): 43.32493702770781,
 (3, 4): 0.5037783375314862,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 0.5050505050505051}

In [29]:
evaluator.worst_group_accuracy

((0, 2), 0.0)

In [30]:
evaluator.average_accuracy

29.97478086995286

In [31]:
evaluator.evaluate_spurious_task()

30.18